In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

filepath_processed = 'C:/Users/Dirk/Google Drive/Dirk/TU/Afstuderen/Afstuderen/Thesis/3. Formalisation/Processed CBS data/'
filepath_raw = 'C:/Users/Dirk/Google Drive/Dirk/TU/Afstuderen/Afstuderen/Thesis/3. Formalisation/Raw CBS data/'
filepath_other = 'C:/Users/Dirk/Google Drive/Dirk/TU/Afstuderen/Afstuderen/Thesis/3. Formalisation/Raw data other/'

In [2]:
#timeit.timeit(pd.read_csv(filepath_processed + 'Total_dataset.csv'))
start = datetime.now()
data =  pd.read_csv(filepath_processed + 'Total_dataset.csv', index_col = 0, 
                    dtype = {'gemeente': str ,'postcode': str ,'huisnummer' : int ,
                             'huisletter': str ,'huisnummertoevoeging': str , 
                             'oppervlakteverblijfsobject' : float ,'pandbouwjaar' : int ,
                             'openbareruimte' : str ,'pandstatus' : str , 'Woningkenmerken': str ,
                             'Energielabel' : str,'leeftijd' : int ,'Age group': str, 
                             'GebruiksOppervlakteklasse' : str,'Bouwjaarklasse' : str,
                             'gasverbruik m3/m2' : float ,'geschat gasverbruik jaarlijks' : float, 
                             'Gemiddelde grootte huishouden' : float , 
                             'elektriciteits verbruik pp' : float , 
                             'geschat elektriciteitsverbruik jaarlijks' : float})

print ('Loading huge dataset took: ' + str((datetime.now()-start).total_seconds()) + ' seconds')

buurten = pd.read_csv(filepath_raw + 'postcode_huisnummer_buurten.csv',delimiter = ';')
wijk = pd.read_csv(filepath_raw + 'wijk2019.csv', delimiter = ';')
buurt = pd.read_csv(filepath_raw + 'buurt2019.csv', delimiter = ';')


C:\Users\Dirk\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Dirk\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Loading huge dataset took: 33.285 seconds


In [4]:
df_total = data

In [9]:
#Count entries per age group, sort so that vensim will understand and fill NaN's with 0's for non existing age groups in municipalities
count_agg_age = df_total.groupby(by = ['gemeente', 'Age group'], sort = False).count()[['openbareruimte']]
count_agg_age = count_agg_age.reset_index().pivot(index='gemeente', columns='Age group', values='openbareruimte')
count_agg_age = count_agg_age[['Age0to9','Age10to19','Age20to29','Age30to39','Age40to49','Age50to59','Age60to69','Age70to79','Age80to89','Age90to99','Age100plus']].fillna(0)

In [10]:
#Save to Excel
count_agg_age.to_csv(filepath_processed + 'Initial Housing Stock.csv')


In [11]:
mean_agg_gas_use = df_total.groupby(by = ['gemeente', 'Age group'], sort = False).mean()[['geschat gasverbruik jaarlijks']]
mean_agg_gas_use = mean_agg_gas_use.reset_index().pivot(index='gemeente', columns='Age group', values='geschat gasverbruik jaarlijks')
mean_agg_gas_use = mean_agg_gas_use[['Age0to9','Age10to19','Age20to29','Age30to39','Age40to49','Age50to59','Age60to69','Age70to79','Age80to89','Age90to99','Age100plus']].ffill(axis = 1)

In [12]:
mean_agg_gas_use.to_csv(filepath_processed + 'Initial Gas use.csv')

In [14]:
average_age_100plus_municipal = df_total.groupby(by = ['gemeente', 'Age group']).mean()[['leeftijd']]
average_age_100plus_municipal = average_age_100plus_municipal.pivot_table(index = "gemeente", columns = "Age group", values = 'leeftijd')[['Age100plus']].fillna(100)

In [15]:
average_age_100plus_municipal.to_csv(filepath_processed + 'Average age 100plus.csv')

In [21]:
# Group by municipality, age group,
mean_elec_age = df_total.groupby(by = ['gemeente', 'Age group'], sort = True).mean()[['geschat elektriciteitsverbruik jaarlijks']]
mean_elec_age = mean_elec_age.pivot_table(index=['gemeente'], columns=['Age group'], values=['geschat elektriciteitsverbruik jaarlijks'])['geschat elektriciteitsverbruik jaarlijks']
mean_elec_age = mean_elec_age[['Age0to9','Age10to19','Age20to29','Age30to39','Age40to49','Age50to59','Age60to69','Age70to79','Age80to89','Age90to99','Age100plus']].sort_index()
mean_elec_age = mean_elec_age.ffill(axis =1)


In [22]:
mean_elec_age.to_csv(filepath_processed + 'Initial Electricity use.csv')